In [11]:
!pip install tensorflow==2.7.0

Defaulting to user installation because normal site-packages is not writeable
     |███████████████████▍            | 296.0 MB 172.9 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 5.8 MB 116.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 132.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 137.1 MB/s eta 0:00:01
     |████████████████████████████████| 13.4 MB 138.0 MB/s eta 0:00:01
     |████████████████████████████████| 463 kB 127.0 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.19.0 requires tensorflow<2.6.0,>=2.5.0, but you have tensorflow 2.7.0 which is incompatible.
tensorflow-io 0.19.0 requires tensorflow-io-gcs-filesystem==0.19.0, but you have tensorflow-io-gcs-filesystem 0.22.0 which is incompa

In [12]:
import pandas as pd
import os
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Modify Data Files

Creates a new file "images.tfrecords" which contains all of the images that had labels in the label sheet. This will only have to be done if you haven't done it already. First unzip "images.zip" into the "../Data Instagram" directory and copy "Labeled_instagram_posts_related_to_covid.xlsx" into the same location. Then run rename_files() to prepare the files for the next step. Run get_labeled_images_raw() to get the matching images and their associated labels. Finally run the cells under "Create TFRecord File" to create the "images.tfrecords" file used by the rest of the notebook. If the "images.tfrecords" file already exists, skip all of this and go to the "5-Fold Cross-Validation" section.

## Get Data Labels

In [11]:
ig_post_data = pd.read_excel("../Data Instagram/Labeled_instagram_posts_related_to_covid.xlsx",
                             usecols="A, N")

## Get Instagram Image Set

Find the corresponding images for the labels we just loaded. Because the file names have extra, unhelpful information at the front, first we remove that information. This allows us to search directly for the image as imagename.jpg. Run rename_files() once if you have not changed the file names already. The images are resized to fit the model inside of get_labeled_images().

In [ ]:
def rename_files():
    # Rename all image files in "Data Instagram", removing the leading integer and underscore
    for filename in os.listdir("../Data Instagram"):
        if filename.endswith(".jpg"):
            new_name = re.sub(r"\d*_", "", filename)
            
            if not os.path.exists("../Data Instagram/" + new_name):
                os.rename("../Data Instagram/" + filename, "../Data Instagram/" + new_name)

rename_files()

In [16]:
def get_labeled_images_raw(image_data):
    images = []
    labels = []
    
    # Get image file name and label
    for post in image_data.itertuples():
        _, imagename, label = post
        
        # Change label to binary
        if label != 1:
            label = 0
        
        # Load and resize image
        file_name = imagename + ".jpg"
        image_file_path = "../Data Instagram/Matching/" + file_name
        if os.path.exists(image_file_path):
            picture = tf.io.read_file(image_file_path)
            #picture = tf.image.decode_jpeg(picture, channels=3)
            #picture = tf.image.resize_with_pad(picture, 480, 480)
            images.append(picture)
            labels.append(label)
    
    return images, labels

In [17]:
ig_images_raw, ig_labels = get_labeled_images_raw(ig_post_data)  # Will store the images we learn with

In [18]:
print(type(ig_images_raw[0]))
print(type(ig_labels[0]))

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'int'>


## Create TFRecord File

Will only need the file "images.tfrecords" from now on.

In [23]:
def create_example(raw_image, label):
    # Create Feature objects from image and label data
    lbl = tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    if isinstance(raw_image, type(tf.constant(0))):
        raw_image = raw_image.numpy()  # Change from EagerTensor
    image_string = tf.train.Feature(bytes_list=tf.train.BytesList(value=[raw_image]))
    
    # Make dictionary for example
    features = {"label": lbl,
                "image_raw": image_string}
    
    example = tf.train.Example(features=tf.train.Features(feature=features))
    return example

In [25]:
output_filename = "../Data Instagram/images.tfrecords"
with tf.io.TFRecordWriter(output_filename) as writer:
    for i in range(len(ig_images_raw)):
        ex = create_example(ig_images_raw[i], ig_labels[i])
        writer.write(ex.SerializeToString())

# 5-Fold Cross-Validation on Instagram Images with Various Models

Run the cross-validation loop once per each model. The results are stored in the Excel sheet "Model_Results", where the averages are calculated across the splits.

## Set Up

### Define Functions

In [13]:
def split_data(cv_round, images, num_images):
    vs = int(num_images / 5) # 20% for validation, 5-fold validation
    valid_start = (cv_round) * vs
    valid_end = (cv_round+1) * vs if cv_round != 4 else -1

    # Split data
    training_ds = images.take(valid_start).concatenate(images.skip(valid_end))
    validation_ds = images.skip(valid_start).take(vs)
    
    return training_ds, validation_ds

In [14]:
def prepare_batch(tds, vds):
    # Batch setup
    batch_size = 32
    training_ds = tds.cache().batch(batch_size).prefetch(buffer_size=10)
    validation_ds = vds.cache().batch(batch_size).prefetch(buffer_size=10)
    
    return training_ds, validation_ds

### Models

In [15]:
def build_model1():
    """
    A basic model with one classifier layer built on top of ResNet50 v2.
    The classification layer uses an average pooling layer followed by a sigmoid activation layer
    for binary classification.
    """
    # Instantiate pre-trained ResNet
    base = keras.applications.ResNet50V2(include_top=False,
                                     weights="imagenet",
                                     input_shape=(480, 480, 3))
    base.trainable = False
    
    # Create classifier
    inputs = keras.Input(shape=(480, 480, 3))
    a = base(inputs, training=False)
    a = keras.layers.GlobalAveragePooling2D()(a)
    outputs = keras.layers.Dense(1, activation="sigmoid")(a)
    classifier = keras.Model(inputs, outputs)
    
    # Compile model
    classifier.compile(optimizer=keras.optimizers.Adam(),
                       loss=keras.losses.BinaryCrossentropy(from_logits=False),
                       metrics=[keras.metrics.BinaryAccuracy(),
                                keras.metrics.Recall(),
                                keras.metrics.Precision()])
    
    return classifier

In [21]:
def build_model2():
    """
    A model with one classifier layer built on top of ResNet50 v2.
    The classification layer uses an average pooling layer followed by a sigmoid activation layer
    for binary classification.
    Adds a data augmentation layer to the input.
    """
    # Instantiate pre-trained ResNet
    base = keras.applications.ResNet50V2(include_top=False,
                                     weights="imagenet",
                                     input_shape=(480, 480, 3))
    base.trainable = False
    
    # Create classifier
    inputs = keras.Input(shape=(480, 480, 3))
    #augmentation = keras.layers.RandomFlip("horizontal")(inputs)
    #augmentation = keras.layers.RandomRotation(0.1)(augmentation)
    a = base(inputs, training=False)
    a = keras.layers.GlobalAveragePooling2D()(a)
    outputs = keras.layers.Dense(1, activation="sigmoid")(a)
    classifier = keras.Model(inputs, outputs)
    
    # Compile model
    classifier.compile(optimizer=keras.optimizers.Adam(),
                       loss=keras.losses.BinaryCrossentropy(from_logits=False),
                       metrics=[keras.metrics.BinaryAccuracy(),
                                keras.metrics.Recall(),
                                keras.metrics.Precision()])
    
    return classifier

In [38]:
def build_model3():
    """
    A model with two classifier layers built on top of ResNet50 v2.
    The classification layer uses an average pooling layer followed by a relu activation layer.
    It uses a sigmoid layer after that for classification.
    """
    # Instantiate pre-trained ResNet
    base = keras.applications.ResNet50V2(include_top=False,
                                     weights="imagenet",
                                     input_shape=(480, 480, 3))
    base.trainable = False
    
    # Create classifier
    inputs = keras.Input(shape=(480, 480, 3))
    #augmentation = keras.layers.RandomFlip("horizontal")(inputs)
    #augmentation = keras.layers.RandomRotation(0.1)(augmentation)
    a = base(inputs, training=False)
    a = keras.layers.GlobalAveragePooling2D()(a)
    a = keras.layers.Dense(2048, activation="relu")(a)
    a = keras.layers.Dense(2048, activation="relu")(a)
    outputs = keras.layers.Dense(1, activation="sigmoid")(a)
    classifier = keras.Model(inputs, outputs)
    
    # Compile model
    classifier.compile(optimizer=keras.optimizers.Adam(),
                       loss=keras.losses.BinaryCrossentropy(from_logits=False),
                       metrics=[keras.metrics.BinaryAccuracy(),
                                keras.metrics.Recall(),
                                keras.metrics.Precision()])
    
    return classifier

### One-time Set Up

In [31]:
def count_positive_labels(old_state, input_element):
    lbl = input_element["label"]
    new_state = old_state + lbl
    return new_state

def process_image(ds_elem):
    image = ds_elem["image_raw"]
    label = ds_elem["label"]
    
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_with_pad(image, 480, 480)
    image = keras.applications.resnet_v2.preprocess_input(image)
    return (image, label)

image_features = {"label": tf.io.FixedLenFeature([], tf.int64),
                  "image_raw": tf.io.FixedLenFeature([], tf.string)}
images_ds = tf.data.TFRecordDataset("../Data Instagram/images.tfrecords")
images_ds = images_ds.map(lambda x: tf.io.parse_single_example(x, image_features))

# Create weight dictionary to offset imbalanced data
num_images = images_ds.reduce(np.int64(0), lambda x, _: x + 1).numpy()
num_pos = images_ds.reduce(np.int64(0), count_positive_labels).numpy()

pos_cls_wgt = (num_images - num_pos) / num_pos
neg_cls_wgt = 1 #(num_images - num_pos) / num_images

cls_wgts_dic = {0: neg_cls_wgt, 1: pos_cls_wgt}
print(cls_wgts_dic)

# Prepare images
images_ds = images_ds.map(process_image)

{0: 1, 1: 5.864693446088795}


## Cross-Validation Loop

The loop records the individual splits' performances in model_results, which is a list of tf.keras.callback.History objects.

In [39]:
model_results = []
for i in range(0, 5):
    # Create the ith data split
    trainset, validset = split_data(i, images_ds, num_images)
    
    # Prepare data
    trainset, validset = prepare_batch(trainset, validset)
    
    # Get model, change the number to get a different model
    model = build_model3()
    
    # Fit model
    result = model.fit(trainset,
                       epochs=15,
                       class_weight=cls_wgts_dic,
                       validation_data=validset)
    model_results.append(result)

Epoch 1/15
82/82 [==============================] - 18s 173ms/step - loss: 1.0441 - binary_accuracy: 0.7421 - recall_20: 0.7630 - precision_20: 0.3360 - val_loss: 0.3309 - val_binary_accuracy: 0.8582 - val_recall_20: 0.8090 - val_precision_20: 0.4898
Epoch 2/15
82/82 [==============================] - 13s 160ms/step - loss: 0.7066 - binary_accuracy: 0.8048 - recall_20: 0.8672 - precision_20: 0.4221 - val_loss: 0.2644 - val_binary_accuracy: 0.8752 - val_recall_20: 0.6292 - val_precision_20: 0.5385
Epoch 3/15
82/82 [==============================] - 13s 159ms/step - loss: 0.5643 - binary_accuracy: 0.8299 - recall_20: 0.8906 - precision_20: 0.4609 - val_loss: 0.2801 - val_binary_accuracy: 0.8659 - val_recall_20: 0.7528 - val_precision_20: 0.5076
Epoch 4/15
82/82 [==============================] - 14s 165ms/step - loss: 0.4971 - binary_accuracy: 0.8426 - recall_20: 0.9115 - precision_20: 0.4828 - val_loss: 0.3056 - val_binary_accuracy: 0.8783 - val_recall_20: 0.6629 - val_precision_20: 0.5

Epoch 4/15
82/82 [==============================] - 13s 159ms/step - loss: 0.5117 - binary_accuracy: 0.8480 - recall_22: 0.9053 - precision_22: 0.4893 - val_loss: 0.4193 - val_binary_accuracy: 0.8012 - val_recall_22: 0.7849 - val_precision_22: 0.4011
Epoch 5/15
82/82 [==============================] - 13s 160ms/step - loss: 0.4702 - binary_accuracy: 0.8680 - recall_22: 0.9368 - precision_22: 0.5274 - val_loss: 0.4504 - val_binary_accuracy: 0.8243 - val_recall_22: 0.7204 - val_precision_22: 0.4323
Epoch 6/15
82/82 [==============================] - 13s 160ms/step - loss: 0.4150 - binary_accuracy: 0.8826 - recall_22: 0.9447 - precision_22: 0.5583 - val_loss: 0.4910 - val_binary_accuracy: 0.8490 - val_recall_22: 0.5806 - val_precision_22: 0.4779
Epoch 7/15
82/82 [==============================] - 14s 172ms/step - loss: 0.3237 - binary_accuracy: 0.9072 - recall_22: 0.9605 - precision_22: 0.6176 - val_loss: 0.5251 - val_binary_accuracy: 0.8721 - val_recall_22: 0.3871 - val_precision_22: 0.5

Epoch 7/15
82/82 [==============================] - 13s 157ms/step - loss: 0.4262 - binary_accuracy: 0.8740 - recall_24: 0.9261 - precision_24: 0.5199 - val_loss: 0.5477 - val_binary_accuracy: 0.8120 - val_recall_24: 0.6612 - val_precision_24: 0.4969
Epoch 8/15
82/82 [==============================] - 13s 162ms/step - loss: 0.3782 - binary_accuracy: 0.8779 - recall_24: 0.9489 - precision_24: 0.5276 - val_loss: 0.5349 - val_binary_accuracy: 0.8305 - val_recall_24: 0.6612 - val_precision_24: 0.5369
Epoch 9/15
82/82 [==============================] - 13s 163ms/step - loss: 0.2881 - binary_accuracy: 0.9060 - recall_24: 0.9631 - precision_24: 0.5947 - val_loss: 0.6084 - val_binary_accuracy: 0.8475 - val_recall_24: 0.5620 - val_precision_24: 0.5965
Epoch 10/15
82/82 [==============================] - 13s 157ms/step - loss: 0.2624 - binary_accuracy: 0.9214 - recall_24: 0.9716 - precision_24: 0.6381 - val_loss: 0.9326 - val_binary_accuracy: 0.7904 - val_recall_24: 0.7107 - val_precision_24: 0.

## Results

Prints the results from model_results. Only shows the last validation metrics for each split.

In [41]:
i = 0
print("Results on validation sets:")
for i in range(5):
    result = model_results[i]
    suffix = '_' + str(i+20)
    
    print("Split " + str(i+1))
    print("Accuracy: {0}\nPrecision: {1}\nRecall: {2}".format(
        result.history['val_binary_accuracy'][-1],
        result.history['val_precision' + suffix][-1],
        result.history['val_recall' + suffix][-1]))
    print("--------------------")

Results on validation sets:
Split 1
Accuracy: 0.8412942886352539
Precision: 0.4507042169570923
Recall: 0.7191011309623718
--------------------
Split 2
Accuracy: 0.8890600800514221
Precision: 0.5714285969734192
Recall: 0.5714285969734192
--------------------
Split 3
Accuracy: 0.8197226524353027
Precision: 0.42941176891326904
Recall: 0.7849462628364563
--------------------
Split 4
Accuracy: 0.8952234387397766
Precision: 0.6607142686843872
Recall: 0.43023255467414856
--------------------
Split 5
Accuracy: 0.8166409730911255
Precision: 0.5061728358268738
Recall: 0.6776859760284424
--------------------
